In [105]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import json
import pprint
import urllib
from urllib.parse import quote
import re

# Used for census.gov data
from census import Census

# Census API Key
from config import census_key
from config import yelp_key

c = Census(census_key, year=2013)

In [160]:
#File with ZIP Code data
file = 'zipcodes_stl2.csv'

# Read all ZIP codes with their Latitude and Longitude for the St. Louis City and St. Louis County area
# When testing use a small set of ZIP Codes: df_zip = df.iloc[0:5,:]
df_zip = pd.read_csv(file)
df_zip.head(5)

,ZIP Code,County,Latitude,Longitude
0,63101,Saint Louis City,38.631551,-90.19300
1,63102,Saint Louis City,38.635200,-90.18702
2,63103,Saint Louis City,38.631451,-90.21415
3,63104,Saint Louis City,38.610701,-90.21362
4,63105,Saint Louis,38.645484,-90.32888


In [69]:
df_census = pd.DataFrame()

for zip in df_zip['ZIP Code']:
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                                  "B19301_001E",
                                  "B17001_002E"), {'for': 'zip code tabulation area:'+str(zip)})
#     print(census_data)
    df_new = pd.DataFrame(census_data)
    df_census = df_census.append(df_new)
# df_census.head()
# Column Reordering
df_census = df_census.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "ZIP Code"})

# Add in Poverty Rate (Poverty Count / Population)
df_census["Poverty Rate"] = 100 * \
    df_census["Poverty Count"].astype(
        int) / df_census["Population"].astype(int)

# Final DataFrame
df_census = df_census[["ZIP Code", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count","Poverty Rate"]]

df_census['ZIP Code'] = pd.to_numeric(df_census['ZIP Code'])
df_census.head()

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,63101,2613.0,30.4,54417.0,42701.0,610.0,23.344814
0,63102,2216.0,33.6,54018.0,32318.0,142.0,6.407942
0,63103,6824.0,30.4,34719.0,30274.0,1340.0,19.636577
0,63104,19520.0,31.2,45498.0,29780.0,5417.0,27.751025
0,63105,17361.0,33.8,86031.0,57408.0,1206.0,6.946604


In [70]:
# Specify the file to write to
output_path = os.path.join("output", "census_data.csv")
df_census.to_csv(output_path)

In [72]:
# Merge in order to link longitude and latitude fields to census data by zipcode 
df_merge = pd.merge(df_census, df_zip, on=('ZIP Code'))
output_path = os.path.join("output", "census_lat_log_data.csv")
df_census.to_csv(output_path)
df_merge.head()

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,County,Latitude,Longitude
0,63101,2613.0,30.4,54417.0,42701.0,610.0,23.344814,Saint Louis City,38.631551,-90.19300
1,63102,2216.0,33.6,54018.0,32318.0,142.0,6.407942,Saint Louis City,38.635200,-90.18702
2,63103,6824.0,30.4,34719.0,30274.0,1340.0,19.636577,Saint Louis City,38.631451,-90.21415
3,63104,19520.0,31.2,45498.0,29780.0,5417.0,27.751025,Saint Louis City,38.610701,-90.21362
4,63105,17361.0,33.8,86031.0,57408.0,1206.0,6.946604,Saint Louis,38.645484,-90.32888


In [75]:
# Get ready to import yelp data
API_KEY=yelp_key

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
MATCH_PATH = '/v3/businesses/matches'
SEARCH_PHONE_PATH = '/v3/businesses/search/phone'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash

In [76]:
def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
#     print(u'Querying {0} ...'.format(url))
#     print(headers)

    response = requests.request('GET', url, headers=headers, params=url_params)
    return response.json() #, my_url

In [96]:
df_businesses = pd.DataFrame()
df_bus_total = pd.DataFrame()
prices = []
dict_total = {}

for index, row in df_merge.iterrows():
    row_zip = row[0]
    row_long = row[9]
    row_lat = row[8]
#     print(index, row_zip, row_lat, row_long)
    url_params={"city":row_lat, 'zip_code':row_zip,'latitude':row_lat,'longitude':row_long}
    dict_businesses = request(API_HOST, SEARCH_PATH, API_KEY, url_params)

    df_businesses['business_id']=[business["id"] for business in dict_businesses['businesses']]
    df_businesses['name']=[business["name"]for business in dict_businesses['businesses']]
    df_businesses['rating']=[business["rating"]for business in dict_businesses['businesses']]
    df_businesses['zip_code']=[business['location']["zip_code"]for business in dict_businesses['businesses']]
    df_businesses['review_count']=[business["review_count"]for business in dict_businesses['businesses']]

    for b in dict_businesses['businesses']:
        try:
            my_price = b['price']
            prices.append(my_price)
        except:
            prices.append('*')
    
#     dict_total.update(dict_businesses)
    
    df_bus_total = df_bus_total.append(df_businesses)
    
df_businesses.head()
# pprint.pprint(dict_businesses)

,business_id,name,rating,zip_code,review_count
0,50QTrfSrow-F2Avsjabl8w,Piekutowski's European Style Sausage,4.0,63107,18
1,siN5JEv_AQj037pwcN32pg,Jerk Soul,4.0,63107,36
2,4P6sn5nKJ2KFYrHUF4-qiA,"The River Lillie, Southern Fusion Cuisine",4.5,63107,13
3,ueJFblv3esW2xjqO7IepGg,Bissell Mansion Restaurant and Murder Mystery ...,3.5,63107,18
4,QnFIWz-CgEVqQ8qhzFrl7A,Cornerstone Cafe,4.0,63107,18


In [97]:
# Add prices info
df_bus_total['price'] = prices
df_bus_total.head(5)

,business_id,name,rating,zip_code,review_count,price
0,8YDJraW_cg5IPTPisPfB-A,City Museum,4.5,63103,1244,*
1,iRIHK8-EwpeffwvoO4nzIA,Broadway Oyster Bar,4.5,63102,1737,$$
2,Mr7Aov2n7wPCpwaUxk8lCw,Mango,4.0,63101,901,$$
3,2BMk_drsikKWslJCXmQtjQ,Rooster,4.0,63101,1754,$$
4,WCdSajl5Q0qywpv7K5jHdQ,Sugarfire Smoke House,4.5,63101,882,$$


In [98]:
# Specify the file to write to
output_path = os.path.join("output", "business_data.csv")
df_bus_total.to_csv(output_path)

In [99]:
# Drop duplicates
df_bus_total = df_bus_total.sort_values('business_id')
df_bus_total = df_bus_total.reset_index(drop=True)
df_bus_unique = df_bus_total.drop_duplicates(['business_id'], keep='first')

df_bus_unique.head(5)
# Optionally use a smaller data set: df_bus_unique_small = df_bus_unique[1:5]

,business_id,name,rating,zip_code,review_count,price
0,-6jVwUuXC1-dyr2YRCEU6Q,Anis Hyderabad Restaurant,4.0,63141,197,$$
2,-W0tnXYJ4WuxLjzmbwlVAQ,Tavolo V,4.0,63112,204,$$
3,-bGw8ARHQucKWEHEkxpGFw,The Daily Bread Bakery & Cafe,4.0,63131,97,$
4,-hto0uJjh8KFQZGc9ZAcuQ,U-City Grill,4.0,63130,181,$
5,-kHHi8y4kaTI6WUVdJSdvQ,Fitz's,3.5,63130,619,$$


In [100]:
# Specify the file to write to
output_path = os.path.join("output", "business_unique_data.csv")
df_bus_unique.to_csv(output_path)

In [ ]:
# Get reviews for each unique business
dict_rev_total = {}
dict_reviews = {}
df_reviews = pd.DataFrame()
df_rev_total = pd.DataFrame()

for index, row in df_bus_unique.iterrows():
    row_bus = row[0]
#     print(row_bus)
    REVIEW_PATH = BUSINESS_PATH + row_bus + '/reviews'
    dict_reviews = request(API_HOST, REVIEW_PATH, API_KEY)
#     pprint.pprint(dict_reviews)
    try:
        df_reviews['business_id']= [row_bus for review in dict_reviews['reviews']]
        df_reviews['review_id']=[review["id"]for review in dict_reviews['reviews']]
        df_reviews['rating']=[review["rating"]for review in dict_reviews['reviews']]
        df_reviews['name']=[review['user']["name"]for review in dict_reviews['reviews']]
        df_rev_total = df_rev_total.append(df_reviews)
    except:
        pass

In [132]:
df_rev_total.head(5)

,index,business_id,review_id,rating,name,short name
0,0,-6jVwUuXC1-dyr2YRCEU6Q,3xNxd1x6ew1QmGDLHxDFdA,5,Maggie Z.,maggie
1,1,-6jVwUuXC1-dyr2YRCEU6Q,CGGd6QCgqoCF67U5GnRaCw,5,Shraddz S.,shraddz
2,2,-6jVwUuXC1-dyr2YRCEU6Q,Vi43zdZWfiZK6jPlGTWTGg,1,Hima R.,hima
3,3,-W0tnXYJ4WuxLjzmbwlVAQ,YWzudmGlyDK5T10tVUk8Kw,5,Colleen S.,colleen
4,4,-W0tnXYJ4WuxLjzmbwlVAQ,GBkWTtokcnP-PMlMEXDLbQ,2,Alyssa G.,alyssa


In [129]:
#COMMENT OUT THE RESET BELOW IF RAN MULTIPLE TIMES!!!!

df_rev_total.reset_index(inplace=True)

# create a list with names per review and take only the name before the 'space' as a first name
df_rev_total['short name']=""
short_name = []
for index, row in df_rev_total.iterrows():
#     print(row)
    short_name = re.split("( )",row[4])
#     print(short_name)
    df_rev_total.iloc[index, 5] = short_name[0].lower()

In [130]:
df_rev_total.head()
# df_rev_total = df_rev_total.drop(['level_0', 'index'], axis=1)

,index,business_id,review_id,rating,name,short name
0,0,-6jVwUuXC1-dyr2YRCEU6Q,3xNxd1x6ew1QmGDLHxDFdA,5,Maggie Z.,maggie
1,1,-6jVwUuXC1-dyr2YRCEU6Q,CGGd6QCgqoCF67U5GnRaCw,5,Shraddz S.,shraddz
2,2,-6jVwUuXC1-dyr2YRCEU6Q,Vi43zdZWfiZK6jPlGTWTGg,1,Hima R.,hima
3,3,-W0tnXYJ4WuxLjzmbwlVAQ,YWzudmGlyDK5T10tVUk8Kw,5,Colleen S.,colleen
4,4,-W0tnXYJ4WuxLjzmbwlVAQ,GBkWTtokcnP-PMlMEXDLbQ,2,Alyssa G.,alyssa


In [131]:
# Specify the file to write to
output_path = os.path.join("output", "review_data.csv")
df_rev_total.to_csv(output_path)

In [134]:
file = 'gender names.csv'
df = pd.read_csv(file)
df = df.set_index('first_name')
df.head(5)

,nameprimary,gender,count
first_name,,,
aa,AA,Not Known,1
aabha,AABHA,Female,1
aabida,AABIDA,Female,1
aad,AAD,Male,1
aadarsh,AADARSH,Male,1


In [137]:
df_rev_total['gender']=""
gender = []
for index, row in df_rev_total.iterrows():
    try:
        gender = df.loc[str(row[5]),'gender']
    #     print(gender)
        df_rev_total.iloc[index, 6] = gender
    except:
        pass
df_rev_total.head()

,index,business_id,review_id,rating,name,short name,gender
0,0,-6jVwUuXC1-dyr2YRCEU6Q,3xNxd1x6ew1QmGDLHxDFdA,5,Maggie Z.,maggie,Female
1,1,-6jVwUuXC1-dyr2YRCEU6Q,CGGd6QCgqoCF67U5GnRaCw,5,Shraddz S.,shraddz,
2,2,-6jVwUuXC1-dyr2YRCEU6Q,Vi43zdZWfiZK6jPlGTWTGg,1,Hima R.,hima,Female
3,3,-W0tnXYJ4WuxLjzmbwlVAQ,YWzudmGlyDK5T10tVUk8Kw,5,Colleen S.,colleen,Female
4,4,-W0tnXYJ4WuxLjzmbwlVAQ,GBkWTtokcnP-PMlMEXDLbQ,2,Alyssa G.,alyssa,Female


In [146]:
# Specify the file to write to
output_path = os.path.join("output", "review_by_gender_data.csv")
df_rev_total.to_csv(output_path)
df_rev_total = df_rev_total.rename(columns={'rating':'reviewer_rating', 'name':'reviewer_full'})

In [150]:
df_total = pd.merge(df_rev_total, df_bus_total,on=('business_id'))
# Somehow renaming columns duplicated rows
# df_total = df_total.drop_duplicates()
df_total.head()

,index,business_id,review_id,reviewer_rating,reviewer_full,short name,gender,name,rating,zip_code,review_count,price
0,0,-6jVwUuXC1-dyr2YRCEU6Q,3xNxd1x6ew1QmGDLHxDFdA,5,Maggie Z.,maggie,Female,Anis Hyderabad Restaurant,4.0,63141,197,$$
2,1,-6jVwUuXC1-dyr2YRCEU6Q,CGGd6QCgqoCF67U5GnRaCw,5,Shraddz S.,shraddz,,Anis Hyderabad Restaurant,4.0,63141,197,$$
4,2,-6jVwUuXC1-dyr2YRCEU6Q,Vi43zdZWfiZK6jPlGTWTGg,1,Hima R.,hima,Female,Anis Hyderabad Restaurant,4.0,63141,197,$$
6,3,-W0tnXYJ4WuxLjzmbwlVAQ,YWzudmGlyDK5T10tVUk8Kw,5,Colleen S.,colleen,Female,Tavolo V,4.0,63112,204,$$
7,4,-W0tnXYJ4WuxLjzmbwlVAQ,GBkWTtokcnP-PMlMEXDLbQ,2,Alyssa G.,alyssa,Female,Tavolo V,4.0,63112,204,$$


In [151]:
# Specify the file to write to
output_path = os.path.join("output", "total_rev_no_lat_long.csv")
df_rev_total.to_csv(output_path)

In [175]:
df_zip = df_zip.rename(columns={'ZIP Code':'zip_code'})
df_total['zip_code'] = pd.to_numeric(df_total['zip_code'])
df_total_zip = pd.merge(df_total, df_zip, on='zip_code', how='outer')
df_total_zip.head()

,index,business_id,review_id,reviewer_rating,reviewer_full,short name,gender,name,rating,zip_code,review_count,price,County,Latitude,Longitude
0,0.0,-6jVwUuXC1-dyr2YRCEU6Q,3xNxd1x6ew1QmGDLHxDFdA,5.0,Maggie Z.,maggie,Female,Anis Hyderabad Restaurant,4.0,63141,197.0,$$,Saint Louis,38.662302,-90.45363
1,1.0,-6jVwUuXC1-dyr2YRCEU6Q,CGGd6QCgqoCF67U5GnRaCw,5.0,Shraddz S.,shraddz,,Anis Hyderabad Restaurant,4.0,63141,197.0,$$,Saint Louis,38.662302,-90.45363
2,2.0,-6jVwUuXC1-dyr2YRCEU6Q,Vi43zdZWfiZK6jPlGTWTGg,1.0,Hima R.,hima,Female,Anis Hyderabad Restaurant,4.0,63141,197.0,$$,Saint Louis,38.662302,-90.45363
3,162.0,4uZUHjSonatwt-02yqJp6w,4FyszXFoCbVZghdgGJyXsw,5.0,Mark H.,mark,Male,Olive Street Cafe,4.0,63141,275.0,$$,Saint Louis,38.662302,-90.45363
4,163.0,4uZUHjSonatwt-02yqJp6w,quwgz4QAbe6ijGL49-nW1A,5.0,Heather R.,heather,Female,Olive Street Cafe,4.0,63141,275.0,$$,Saint Louis,38.662302,-90.45363
